# PathoScopy: Medical Knowledge Retrieval Chatbot

“A chatbot that examines medical literature and provides accurate, source-cited answers using RAG and LLMs."

Pathoscopy leverages a Retrieval-Augmented Generation pipeline to answer medical questions from a curated corpus of abstracts while citing sources for transparency.

## Install & Import dependencies

In [1]:
# Run to install required packages
!pip install datasets transformers
!pip install --upgrade langchain langchain-community langchain-core langchain-text-splitters
!pip install sentence-transformers
!pip install faiss-cpu
!pip install -q transformers accelerate bitsandbytes
!pip install -q langchain_community transformers sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 38.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.


In [2]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import List
import matplotlib.pyplot as plt
import torch

## Load Dataset

data source: https://www.kaggle.com/datasets/chaitanyakck/medical-text

In [3]:
# Load your knowledge base text file
with open('/kaggle/input/medical-abstracts/train.txt', 'r') as f:
    data = f.read()

print(f"File size: {len(data)} characters")
print("First 500 characters:\n", data[:500])

File size: 17811683 characters
First 500 characters:
 4	Catheterization laboratory events and hospital outcome with direct angioplasty for acute myocardial infarction To assess the safety of direct infarct angioplasty without antecedent thrombolytic therapy, catheterization laboratory and hospital events were assessed in consecutively treated patients with infarctions involving the left anterior descending (n = 100 patients), right (n = 100), and circumflex (n = 50) coronary arteries. The groups of patients were similar for age (left anterior desce


## Document Preparation & Text Splitting

In [4]:
# Split raw text into manageable chunks for embedding
from langchain_core.documents import Document as LangchainDocument
from langchain_text_splitters import RecursiveCharacterTextSplitter

RAW_KNOWLEDGE_BASE = LangchainDocument(page_content=data)

MARKDOWN_SEPARATORS = [
    "\n#{1,6} ", "```\n", "\n\\*\\*\\*+\n", "\n---+\n", "\n___+\n",
    "\n\n", "\n", " ", ""
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=MARKDOWN_SEPARATORS
)

docs_processed = text_splitter.split_documents([RAW_KNOWLEDGE_BASE])
print(f"Original document split into {len(docs_processed)} chunks")

2025-11-09 20:05:24.990678: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762718725.386613      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762718725.516948      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Original document split into 25860 chunks


## Embeddings & Vector Store Setup (FAISS)

In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Initialize embeddings model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
print("Embeddings model loaded!")

# Path to save/load vector store

vectorstore_path = "./faiss_index"

# Load existing FAISS index or create if it doesn't exist
try:
    vectorstore = FAISS.load_local(vectorstore_path, embeddings, allow_dangerous_deserialization=True)
    print("Vector store loaded successfully!")
except:
    vectorstore = FAISS.from_documents(documents=docs_processed, embedding=embeddings)
    vectorstore.save_local(vectorstore_path)
    print(f"Vector store created and saved with {len(docs_processed)} documents!")

/tmp/ipykernel_48/1096709084.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings model loaded!
Vector store created and saved with 25860 documents!


### Load LLM

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "microsoft/Phi-3-mini-4k-instruct"  # Change if desired

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)

llm_model = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.4,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500
)
print("LLM loaded!")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0


LLM loaded!


## Prompt Template

In [7]:
def build_prompt_cited(context_texts: list, user_question: str) -> str:
    """
    Generates a prompt instructing the model to:
    - Answer using only the retrieved medical abstracts
    - Be concise (3–5 sentences)
    - Cite sources for every factual statement
    - Respond "I don't know" if context is insufficient
    """
    context_combined = "\n".join([f"[Source {i+1}]: {txt}" for i, txt in enumerate(context_texts)])
    prompt = f"""
You are a medical assistant. Answer the user's question using ONLY the context below.
Be concise (3–5 sentences) and cite sources for every factual statement using [Source #].
If the context does not contain the answer, respond: "I don't know. No relevant information in the context."

Context:
{context_combined}

Question: {user_question}

Answer:
"""
    return prompt

## Chatbot Loop (RAG in Action)

User Question → Embed Query → FAISS Vector Search → Retrieve Documents → LLM Prompt → Answer with Sources


## Example Chat
**User:** What is Cardiogenic shock?  
**AI Response:** Cardiogenic shock occurs when there is insufficient circulation due to heart dysfunction after events like myocardial infarction…  
**Sources used:** Source 3, Source 4


In [8]:
print("\n=== PathoScopy AI Chatbot Ready! Type 'exit' to quit ===")
while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chatbot...")
        break

    # Step 1: Embed the query
    query_vector = embeddings.embed_query(user_input)

    # Step 2: Retrieve top relevant documents (increase k for robustness)
    top_docs = vectorstore.similarity_search_by_vector(query_vector, k=5)

    # Step 3: Check for empty retrieval
    if not top_docs or all(len(doc.page_content.strip()) == 0 for doc in top_docs):
        print("AI response: I don't know. No relevant information in the context.")
        continue

    # Step 4: Optional: filter out low-relevance docs based on semantic similarity
    # (requires vectorstore supporting similarity scores; otherwise rely on top-k)

    # Step 5: Build prompt
    final_prompt = build_prompt_cited([doc.page_content for doc in top_docs], user_input)

    # Step 6: Generate answer
    answer = llm_model(final_prompt)[0]['generated_text']

    print("\nPathoScopy response:", answer)

    # Step 7: Show sources used (preview first 200 characters)
    print("\nSources used:")
    for i, doc in enumerate(top_docs, start=1):
        preview = doc.page_content[:200].replace("\n", " ")
        print(f"- Source {i}: {preview}...")


=== PathoScopy AI Chatbot Ready! Type 'exit' to quit ===


User:  What is Cardiogenic shock?



PathoScopy response: Cardiogenic shock occurs when there is severe dysfunction of the heart muscle leading to insufficient perfusion of organs due to decreased pump action by the heart itself ([Source 3]). It often follows events like significant myocardial damage such as seen after an Acute Myocardial Infarction where it can be associated with high mortality rates if rapid intervention isn’t provided ([Sources 1 & Source 5]). Factors influencing its occurrence include preexisting conditions like prior myocardial infarctions, elevated levels of certain biomarkers indicative of cell death like Creatine Kinase Concentrations, Blood Urea Nitrogen, etc., along with radiological signs evident via Chest X-Ray showing enlarged heart silhouette known as 'cardiothoracic ratio.' These factors combined help healthcare providers assess risks and manage patient outcomes effectively ([Sources 1 - 5]).

Sources used:
- Source 1: established in all three patients (atrial fibrillation, polymorphic ven

User:  What is the survival rate for cardiogenic shock?



PathoScopy response: The provided text contains no direct data on overall survival rates specifically related to all cases or conditions involving cardiogenic shock; it discusses outcomes from studies focused primarily on treatments like implantable defibrillators (AICD) and acute percutaneous transluminal coronary angioplasty within specific patient populations suffering from various forms of arrhythmias or post-myocardial infarction scenarios. To accurately address your query about general survival rates associated directly with cardiogenic shock as mentioned across multiple documents would require additional explicit source material detailing such statistics which isn’t available here. Therefore, I cannot provide this particular statistic based solely on these given texts without further research into up-to-date comprehensive databases or literature focusing explicitly on current survival metrics linked to cardiogenic shock itself. For precise figures regarding mortality due to car

User:  Can cardiogenic shock occur without a heart attack?



PathoScopy response: Cardiogenic shock can indeed occur due to causes other than a heart attack as indicated by Source 2 where it describes a case involving 'acute myocarditis'. Myocarditis is inflammation of the heart muscle that could lead to impaired pumping function resulting in cardiogenic shock even when there isn’t any blockage in the coronary arteries typically seen in heart attacks. However, this condition would still require immediate attention since source also highlights its potential severity leading possibly to life threatening situations like cardiac arrest if undetected or improperly managed. Therefore while most instances might involve some form of damage related directly through blocked vessels such as those caused by clots forming at sites within these conduits commonly associated with MIs - they aren't limited strictly only towards them being present before onset occurs nor necessarily always indicative upon diagnosis alone either given their overlapping symptomato

User:  exit


Exiting chatbot...
